In [ ]:
import geopandas as gpd
from shapely import delaunay_triangles, MultiPolygon, GeometryCollection
from shapely.geometry.multilinestring import MultiLineString

from gen_planner.python.src import GenPlanner

polywholes = gpd.read_file('poly_holes.geojson').to_crs(32636)
# polywholes.explore()

poly_w_holes = polywholes.geometry.iloc[0]

# def poly_with_holes_to_multipoly(polygon):

# m1 = polywholes.explore(color='red')
# # gpd.GeoDataFrame(geometry=[delaunay_triangles(poly_w_holes)],crs=polywholes.crs).explode().explore(m=m1)
# m1

In [ ]:
poly_w_holes = shitpoly.geometry.iloc[0]

In [ ]:
from shapely import LineString
from shapely.geometry import Polygon,MultiPolygon,GeometryCollection
from scipy.spatial.distance import cdist
import numpy as np
from shapely.ops import unary_union
from shapely.ops import nearest_points,polygonize
from app.gen_planner.python.src.utils import geometry_to_multilinestring




inner_geoms = [Polygon(ring) for ring in poly_w_holes.interiors]

while len(inner_geoms)>0:
    print(len(inner_geoms))
    outside_poly = Polygon(poly_w_holes.exterior)
    lines = []
    for i in range(len(inner_geoms)):
        all_but_cur = inner_geoms.copy()
        poly = all_but_cur.pop(i)
        lines.append(LineString(nearest_points(poly, GeometryCollection(all_but_cur + [poly_w_holes.exterior]))).buffer(0.1,resolution=1).exterior)
        
    polygons = list(polygonize(unary_union([geometry_to_multilinestring(poly_w_holes)] + lines)))
    repr_point = poly_w_holes.representative_point()
    for poly in polygons:
        if poly.contains(repr_point):
            poly_w_holes = poly
            break
    inner_geoms = [Polygon(ring) for ring in poly_w_holes.interiors]


In [ ]:
from app.gen_planner.python.src.utils import geometry_to_multilinestring
from shapely import LineString
from shapely.ops import nearest_points
from shapely.ops import unary_union, polygonize

buffered_poly_w_holes = poly_w_holes.buffer(0.1)
lines = [LineString(nearest_points(poly, buffered_poly_w_holes.exterior)).buffer(0.1, resolution=1).exterior for poly in
         buffered_poly_w_holes.interiors]
polygons = list(polygonize(unary_union([geometry_to_multilinestring(poly_w_holes)] + lines)))
repr = poly_w_holes.representative_point()
for poly in polygons:
    if poly.contains(repr):
        result = poly
        break
result

In [ ]:
from shapely import line_merge, MultiLineString

splitted_lines = gpd.GeoSeries(
    pd.Series(zone_roads.to_crs(32636).geometry.apply(geometry_to_multilinestring).explode().apply(
        explode_linestring)).explode(
        ignore_index=True), crs=32636).normalize().set_precision(0.00000001).unique()

# splitted_lines = zone_splitted.to_crs(32636).geometry.apply(geometry_to_multilinestring).explode().to_list()

lines = line_merge(MultiLineString(splitted_lines))
#
# gpd.GeoDataFrame(geometry=lines,crs=32636).explode(ignore_index=True).reset_index().explore(column='index')

print(len(lines.geoms))
print(len(splitted_lines))

In [ ]:
import geopandas as gpd
from app.gen_planner.python.src import GenPlanner, func_zones, terr_zones

zone = gpd.read_file('territory.geojson')
zone_roads = gpd.read_file('roads.geojson')
exclude = gpd.read_file('exclude.geojson')
gp = GenPlanner(zone, zone_roads, exclude,simplify_geometry=True,simplify_value=10,dev_mod=True)
generated_zones, generated_roads = gp.features2terr_zones2blocks(func_zones.residential_func_zone)
generated_zones.explore(tiles='Cartodb Positron', column='territory_zone', cmap='Dark2')

In [ ]:
from app.gen_planner.python.src import territory_splitter
from app.gen_planner.python.src.utils import geometry_to_multilinestring, explode_linestring
import geopandas as gpd
import pandas as pd

zone = gpd.read_file('../scenarios_data/project Шлиссельбург.geojson')
zone_roads = gpd.read_file('../scenarios_data/roads_shlis.geojson')

In [ ]:
from app.gen_planner.python.src import GenPlanner, func_zones, terr_zones
from shapely import Point

zones_ratio_dict = {'right_lower': 0.225, 'right_upper': 0.225, 'left_upper': 0.225, 'left_lower': 0.225,
                    'middle_lower': 0.1, 'middle_upper': 0.1, 'free_zone1': 0.1, 'free_zone2': 0.1}
fixed_zones = gpd.GeoDataFrame()
# middle_lower = Point(31.015, 59.911)
middle_upper = Point(31.022, 59.922)
right_upper = Point(31.046, 59.922)
right_lower = Point(31.043, 59.911)
left_upper = Point(30.999, 59.927)
left_lower = Point(30.993, 59.912)

fixed_2zones = gpd.GeoDataFrame(
    {'fixed_zone': [terr_zones.transport_terr, terr_zones.recreation_terr]},
    geometry=[left_lower, middle_upper],
    crs=4326)

fixed_4zones = gpd.GeoDataFrame(
    {'fixed_zone': [terr_zones.transport_terr, terr_zones.recreation_terr, terr_zones.special_terr,
                    terr_zones.business_terr]},
    geometry=[right_upper, right_lower, left_upper, left_lower],
    crs=4326)

fixed_zones = fixed_4zones

territory = zone.copy()

generated_zones, generated_roads = GenPlanner(zone, zone_roads,dev_mod=True).features2terr_zones2blocks(
    funczone=func_zones.residential_func_zone,fixed_terr_zones=fixed_zones)
# gp= GenPlanner(zone, zone_roads,exclude = None,dev_mod=True)
# generated_zones, generated_roads = gp.features2terr_zones2blocks(
#     funczone=func_zones.residential_func_zone)

In [ ]:
import pandas as pd

m1 = pd.concat([generated_zones,
                fixed_zones.rename(columns={'fixed_zone': 'territory_zone'}).to_crs(generated_zones.crs)]).explore(
    tiles='Cartodb Positron', column='territory_zone', cmap='Dark2')
m1

In [ ]:
generated_roads.explore(column='road_lvl')

In [ ]:
GenPlanner(zone, zone_roads, exclude, dev_mod=True).territory_to_work_with.explore()

In [ ]:
generated_roads.explore(column='road_lvl', tiles='Cartodb Positron')